In [1]:
import numpy as np
import cv2 as cv
import mediapipe as mp
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import utils
from sklearn.tree import DecisionTreeClassifier

from sklearn import *
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, mean_absolute_error,accuracy_score,confusion_matrix
import csv

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mPose = mp.solutions.pose

In [2]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = (radians * 180.0 / np.pi)

    if angle > 180.0:
        angle -= 360.0
    return angle 
  

In [3]:
#mediapipe setup
#cap = cv.VideoCapture("Split\yer.mp4")
with mPose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    # while cap.isOpened():
    #     ret, frame = cap.read()
    
    img= cv.imread("Dataset\Image_3.jpg")

    imag = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    imag.flags.writeable = False

    results = pose.process(imag)

    imag.flags.writeable = True
    imag = cv.cvtColor(imag, cv.COLOR_RGB2BGR)

    imag = cv.resize(imag, dsize=(700,500))

    mp_drawing.draw_landmarks(imag, results.pose_landmarks, mPose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
    
    try:
        landmarks = results.pose_landmarks.landmark 

        #coordinates    
        rhip = (landmarks[mPose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mPose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mPose.PoseLandmark.RIGHT_HIP.value].z)
        rknee = landmarks[mPose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mPose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mPose.PoseLandmark.RIGHT_KNEE.value].z                        

        lhip = (landmarks[mPose.PoseLandmark.LEFT_HIP.value].x,landmarks[mPose.PoseLandmark.LEFT_HIP.value].y,landmarks[mPose.PoseLandmark.LEFT_HIP.value].z)
        lknee = landmarks[mPose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mPose.PoseLandmark.LEFT_KNEE.value].y,landmarks[mPose.PoseLandmark.LEFT_KNEE.value].z                         
        
        hip = [(rhip[0] + lhip[0]) / 2, (rhip[1] + lhip[1]) / 2]
        angle = calculate_angle(lknee, hip, rknee)

        if 160.0 < angle < 180.0:
            deduct = ('Deduct: 0.10')
        
        elif 135.0 < angle < 160.0:    
            deduct = ('Deduct: 0.30')
        
        elif 0< angle < 135.0:
            deduct = ('No DV')
  
        else: 
            deduct = ('No deduct')
        
        cv.putText(imag, deduct, (0,85),
                cv.FONT_HERSHEY_SIMPLEX, 0.75, (220,220,220), 2, cv.LINE_AA)
                 
        #visualize angle 
        cv.putText(imag, str(angle),
                tuple(np.multiply(hip, [0, 80]).astype(int)),
                cv.FONT_HERSHEY_SIMPLEX, 0.75, (220,220,220), 2, cv.LINE_AA)
               
    except:
        pass

    #mp_drawing.plot_landmarks(results.pose_world_landmarks, mPose.POSE_CONNECTIONS)

    
    cv.imshow('split',imag)
        
    if cv.waitKey(10) & 0xFF == ord('q'):
        cv.destroyAllWindows()
            #break
    

cv.waitKey(0)
#cap.release()
cv.destroyAllWindows()

In [4]:
results.pose_landmarks